## Kaggle Titanic Competition
#### Top Score: 0.8010 (i.e., a model with efficiency of 80%)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
X = pd.read_csv('/kaggle/input/titanic/train.csv')
X_final = pd.read_csv('/kaggle/input/titanic/test.csv')
y = X.Survived
# X_final = X_final.loc[:,X_final.columns.drop(['SibSp','Parch','Embarked'])]
# X = X.loc[:,X.columns.drop(['Survived','SibSp','Parch','Embarked'])]
# X_final = X_final.loc[:,X_final.columns.drop(['Parch','Embarked'])]
# X = X.loc[:,X.columns.drop(['Name','Survived','Parch','Embarked','SibSp','Cabin'])]
# X_final = X_final.loc[:,X_final.columns.drop(['Name','Parch','Embarked','SibSp','Cabin'])]
# X_final =X_final.loc[:,X_final.columns.drop(['Cabin'])]
X = X.loc[:,X.columns.drop(['Survived'])]
X_train, X_test, y_train, y_test = train_test_split(X,y)
ids = X_final.PassengerId
X_train

def adder(dataset):
    Is_women_or_child = [x == 'female' or y < 18 for x,y in zip(dataset.Sex,dataset.Age)]
    dataset['is_women_or_child'] = Is_women_or_child
adder(X_train)
adder(X_test)
adder(X_final)
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_women_or_child
533,534,3,"Peter, Mrs. Catherine (Catherine Rizk)",female,NaN,0,2,2668,22.3583,NaN,C,True
730,731,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,B5,S,True
84,85,2,"Ilett, Miss. Bertha",female,17.0,0,0,SO/C 14885,10.5000,NaN,S,True
497,498,3,"Shellard, Mr. Frederick William",male,NaN,0,0,C.A. 6212,15.1000,NaN,S,False
878,879,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,False
...,...,...,...,...,...,...,...,...,...,...,...,...
101,102,3,"Petroff, Mr. Pastcho (""Pentcho"")",male,NaN,0,0,349215,7.8958,NaN,S,False
543,544,2,"Beane, Mr. Edward",male,32.0,1,0,2908,26.0000,NaN,S,False
642,643,3,"Skoog, Miss. Margit Elizabeth",female,2.0,3,2,347088,27.9000,NaN,S,True
573,574,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.7500,NaN,Q,True


In [3]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
# from sklearn.preprocessing import OrdinalEncoder

# ordinal_encoder = OrdinalEncoder()
# X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
# X_test[object_cols] = ordinal_encoder.fit_transform(X_test[object_cols])
# X_final[object_cols] = ordinal_encoder.fit_transform(X_final[object_cols])

Categorical variables:
['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [4]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))
OH_cols_final = pd.DataFrame(OH_encoder.transform(X_final[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index
OH_cols_final.index = X_final.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)
num_X_final = X_final.drop(object_cols,axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
OH_X_final = pd.concat([num_X_final, OH_cols_final], axis=1)




In [5]:
from sklearn.impute import SimpleImputer 

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(OH_X_test))
imputed_X_final = pd.DataFrame(my_imputer.transform(OH_X_final))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [6]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.tree import DecisionTreeRegressor

In [7]:
model = RandomForestRegressor(n_estimators = 75, random_state =27,max_leaf_nodes =30)
model.fit(imputed_X_train,y_train)
predictorForest = model.predict(imputed_X_test)
print(mean_squared_error(y_test,predictorForest))
comparer = (mean_squared_error(y_test,predictorForest))



0.12406524433989112


In [8]:

XGBModel = XGBRegressor(n_estimators = 4, learning_rate = 0.5 + 1*0.01,max_leaves = 2, early_stopping_rounds = 2, max_depth = 5)
XGBModel.fit(imputed_X_train,y_train,
         eval_set = [(imputed_X_test,y_test)],
         verbose = False)

XGBPredictor = XGBModel.predict(imputed_X_test)
print(mean_squared_error(y_test,XGBPredictor))



0.12657229401207787


In [9]:

finPred_RandomForest = model.predict(imputed_X_final)
finPred_XGB = XGBModel.predict(imputed_X_final)
# print(finPred_RandomForest)
# print(finPred_XGB)
submissionList = [int(x > 0.75) for x in finPred_RandomForest]


In [10]:
submitter = pd.DataFrame().assign(PassengerId = ids, Survived = submissionList)
submitter = submitter.iloc[: ,0:]
submitter.to_csv("submission.csv",index = False)